# Pytorch 101 - 004
#### *Date* : 2019.05.01
#### *Auther* :`Jen-Huan Hu`

### Comfirm cuda device via ***torch.cuda*** commands :

In [98]:
import torch
print("Check if cuda device exists via 'torch.cuda.device_count()' : {}".format( torch.cuda.device_count() ))
print("Check current device via 'torch.cuda.current_device()' : {}".format( torch.cuda.current_device() ))
print("Check cuda device #1 via 'torch.cuda.device(0)' : {}".format( torch.cuda.device(0) ))
print("Check cuda device #1 name via 'torch.cuda.get_device_name(0)' : {}".format( torch.cuda.get_device_name(0)) )
print("Try 'torch.cuda.get_device_capability(0)' : {}".format( torch.cuda.get_device_capability(0) ))
print("Try 'torch.cuda.get_device_properties(0)' : {}".format( torch.cuda.get_device_properties(0) ))

Check if cuda device exists via 'torch.cuda.device_count()' : 1
Check current device via 'torch.cuda.current_device()' : 0
Check cuda device #1 via 'torch.cuda.device(0)' : <torch.cuda.device object at 0x000002020101BB00>
Check cuda device #1 name via 'torch.cuda.get_device_name(0)' : GeForce GTX 1060 with Max-Q Design
Try 'torch.cuda.get_device_capability(0)' : (6, 1)
Try 'torch.cuda.get_device_properties(0)' : _CudaDeviceProperties(name='GeForce GTX 1060 with Max-Q Design', major=6, minor=1, total_memory=6144MB, multi_processor_count=10)


### For any torch vector, make it to reside in either cpu or cuda device via ***<torch.tensor>.to()***

In [99]:
import numpy as np

device = 'cuda' if torch.cuda.device_count() else 'cpu'
print('torch device using {}'.format( device ))

a = torch.tensor(np.arange(-10., 10., 2.5))
b = torch.randn((5, 3))
c = torch.tensor(np.array([1,2,3,4,5]), device = device)
print("a = %s" % a)
print("a.device = %s" % a.device)
print("b = %s" % b)
print("b.device = %s" % b.device)
a = a.to('cuda')
print("a = %s" % a)
print("a.device = %s" % a.device)
print("c.device = %s" % c.device)

torch device using cuda
a = tensor([-10.0000,  -7.5000,  -5.0000,  -2.5000,   0.0000,   2.5000,   5.0000,
          7.5000], dtype=torch.float64)
a.device = cpu
b = tensor([[ 0.3226, -0.0261,  0.8072],
        [-0.6731, -0.8248, -0.8590],
        [-2.0269,  1.9285,  1.1771],
        [-2.6445,  1.5218,  1.6197],
        [-0.7643,  0.4250, -1.4369]])
b.device = cpu
a = tensor([-10.0000,  -7.5000,  -5.0000,  -2.5000,   0.0000,   2.5000,   5.0000,
          7.5000], device='cuda:0', dtype=torch.float64)
a.device = cuda:0
c.device = cuda:0


## Now let's try 2D convolution : 
### 1. *torch.conv2d*
with kernel and bias provided from outside
### 2. *torch.nn.Conv2d*
***CLASS torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)***  
directly produce a callable functor with <torch.nn.Conv2d>.weight and <torch.nn.Conv2d>.bias as kernel and bias
### 3. *torch.nn.functional.conv2d*
***torch.nn.functional.conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1, padding_mode='zeros') → Tensor***  
provide kernel and bias from outside
### Follow the shape convention :
#### Input: (N, C_{in}, H_{in}, W_{in})
#### Output: (N, C_{out}, H_{out}, W_{out})

In [100]:
help(torch.nn.Conv2d)

Help on class Conv2d in module torch.nn.modules.conv:

class Conv2d(_ConvNd)
 |  Applies a 2D convolution over an input signal composed of several input
 |  planes.
 |  
 |  In the simplest case, the output value of the layer with input size
 |  :math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
 |  can be precisely described as:
 |  
 |  .. math::
 |      \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
 |      \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)
 |  
 |  
 |  where :math:`\star` is the valid 2D `cross-correlation`_ operator,
 |  :math:`N` is a batch size, :math:`C` denotes a number of channels,
 |  :math:`H` is a height of input planes in pixels, and :math:`W` is
 |  width in pixels.
 |  
 |  * :attr:`stride` controls the stride for the cross-correlation, a single
 |    number or a tuple.
 |  
 |  * :attr:`padding` controls the amount of implicit zero-pad

In [101]:
x = torch.relu( torch.randn((4,5,3)) * 10 )
x = x.type(torch.int32)
print('x = %s' % x)

# reshape to N, C, H, W 
x = x.permute(2, 0, 1) # or use <tensor>.transpose(<axis-1>, <axis-2>) ?
print('After permute : x = %s' % x)

x = tensor([[[ 0,  0,  0],
         [ 9, 16,  2],
         [ 6,  6,  6],
         [ 1,  1,  0],
         [ 0,  0, 12]],

        [[10,  0,  0],
         [ 0,  1, 11],
         [ 8, 19,  0],
         [ 0,  4,  0],
         [14, 14,  9]],

        [[ 9,  1, 21],
         [ 7,  0,  2],
         [ 0, 19,  0],
         [ 0,  0, 10],
         [ 0, 11,  0]],

        [[ 6,  0,  8],
         [ 0,  0,  0],
         [ 0,  0,  8],
         [ 2,  1,  1],
         [ 0,  0,  0]]], dtype=torch.int32)
After permute : x = tensor([[[ 0,  9,  6,  1,  0],
         [10,  0,  8,  0, 14],
         [ 9,  7,  0,  0,  0],
         [ 6,  0,  0,  2,  0]],

        [[ 0, 16,  6,  1,  0],
         [ 0,  1, 19,  4, 14],
         [ 1,  0, 19,  0, 11],
         [ 0,  0,  0,  1,  0]],

        [[ 0,  2,  6,  0, 12],
         [ 0, 11,  0,  0,  9],
         [21,  2,  0, 10,  0],
         [ 8,  0,  8,  1,  0]]], dtype=torch.int32)


In [102]:
# expand axis to make first dimension as batch axis
x = x[:,:,:, None]
print(x.shape)
x = x.permute(3, 0, 1, 2)

print('Add new axis : x = %s, has shape %s' % (x, x.shape))

torch.Size([3, 4, 5, 1])
Add new axis : x = tensor([[[[ 0,  9,  6,  1,  0],
          [10,  0,  8,  0, 14],
          [ 9,  7,  0,  0,  0],
          [ 6,  0,  0,  2,  0]],

         [[ 0, 16,  6,  1,  0],
          [ 0,  1, 19,  4, 14],
          [ 1,  0, 19,  0, 11],
          [ 0,  0,  0,  1,  0]],

         [[ 0,  2,  6,  0, 12],
          [ 0, 11,  0,  0,  9],
          [21,  2,  0, 10,  0],
          [ 8,  0,  8,  1,  0]]]], dtype=torch.int32), has shape torch.Size([1, 3, 4, 5])


In [103]:
import torch.nn as nn
# torch.conv2d
# torch.nn.Conv2d
# torch.nn.functional.conv2d
# torch.nn.functional.conv_transpose2d

# torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
x = x.type(torch.float32)
# x = x.to(device)
c1 = nn.Conv2d(x.shape[1], 4, 3, stride=(1,1), padding = (1,1))
x2 = c1( x )
print("x2 = %s of shape %s" % (x2, x2.shape))
print(c1.weight, c1.bias)

x2 = tensor([[[[ 2.4120,  3.5910,  3.0876,  1.3106,  3.2127],
          [ 5.4936,  1.0632,  0.0268,  3.5369,  0.0541],
          [ 0.5174,  2.0886,  0.3250,  4.2243, -2.9563],
          [-2.1012,  1.2165,  0.1053, -2.6177, -2.0330]],

         [[ 2.0123, -0.4714, -6.2841, -2.0067, -1.8003],
          [ 3.1480, -5.8397, -5.4140,  1.1358, -2.0245],
          [-0.9431,  0.8080,  0.2511, -8.0247,  0.5102],
          [ 2.0007,  3.4866, -2.7398,  0.1506,  1.2076]],

         [[-3.6319,  1.8007,  4.2506, -0.3342,  1.3977],
          [ 1.0782,  6.3760, -1.0772, -5.9988,  3.2720],
          [ 1.1935, -6.4391,  2.8020,  1.7751, -0.2542],
          [ 0.2959, -1.4613,  1.7441, -1.6855,  2.5792]],

         [[-4.1246, -3.1700,  1.4920, -3.4762,  2.1937],
          [-0.2022, -4.8056, -1.5206, -3.6980,  5.7713],
          [ 1.0800, -5.1590, -0.4570, -2.8144, -1.2044],
          [ 3.6611, -1.4142, -2.2824,  1.6976,  0.5922]]]],
       grad_fn=<ThnnConv2DBackward>) of shape torch.Size([1, 4, 4, 5])
Par

In [104]:
help(torch.conv2d)

Help on built-in function conv2d:

conv2d(...)
    conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1) -> Tensor
    
    Applies a 2D convolution over an input image composed of several input
    planes.
    
    See :class:`~torch.nn.Conv2d` for details and output shape.
    
    .. include:: cudnn_deterministic.rst
    
    Args:
        input: input tensor of shape :math:`(\text{minibatch} \times \text{in\_channels} \times iH \times iW)`
        weight: filters of shape :math:`(\text{out\_channels} \times \frac{\text{in\_channels}}{\text{groups}} \times kH \times kW)`
        bias: optional bias tensor of shape :math:`(\text{out\_channels})`. Default: ``None``
        stride: the stride of the convolving kernel. Can be a single number or a
          tuple `(sH, sW)`. Default: 1
        padding: implicit zero paddings on both sides of the input. Can be a
          single number or a tuple `(padH, padW)`. Default: 0
        dilation: the spacing between kernel

In [105]:
k_out_channel, k_in_channel, k_w, k_h = 5, x.shape[1], 3, 3
kernel = torch.zeros((k_out_channel, k_in_channel, k_w, k_h))
kernel = torch.nn.init.xavier_normal_(kernel)
print("kernel = %s" % kernel)
print(kernel.shape)

x3 = torch.conv2d( x, kernel )
print("x3 = %s of shape %s" % (x3, x3.shape))

import torch.nn.functional as F
# torch.nn.functional.conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1, padding_mode='zeros') → Tensor
x4 = F.conv2d(x, kernel, stride = (2, 2))
print("x4 = %s of shape %s" % (x4, x4.shape))

kernel = tensor([[[[-0.0089, -0.1547,  0.0081],
          [ 0.0798, -0.1043,  0.3239],
          [ 0.2929,  0.0033, -0.0281]],

         [[ 0.0370, -0.4227,  0.3668],
          [-0.0281, -0.0059, -0.1393],
          [ 0.2590,  0.0742, -0.0212]],

         [[ 0.2148,  0.1024,  0.0432],
          [-0.0022,  0.1501,  0.0541],
          [ 0.0455, -0.1562, -0.2283]]],


        [[[-0.1780, -0.1513, -0.0897],
          [-0.0051, -0.0051,  0.2008],
          [-0.0097,  0.0056, -0.1096]],

         [[-0.4929, -0.0220, -0.1346],
          [-0.0460, -0.0052,  0.0097],
          [-0.0675,  0.1066,  0.2846]],

         [[ 0.1029, -0.0240,  0.0096],
          [-0.0547, -0.0583, -0.3137],
          [ 0.1493,  0.1693,  0.2812]]],


        [[[-0.0091,  0.0625, -0.0642],
          [-0.0514, -0.0309,  0.0893],
          [ 0.0120,  0.0693,  0.1885]],

         [[ 0.0630, -0.0112, -0.1684],
          [-0.1805,  0.0754, -0.2252],
          [-0.1293,  0.1179, -0.3479]],

         [[ 0.4887, -0.4296, -0.090